In [1]:
from typing import TypedDict, List, Dict, Union, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
import operator

In [2]:
def add(old, new):
    return old + new

class State(TypedDict):
    words: List[str]
    output: Annotated[List[Dict[str, Union[str, int]]], add]

In [3]:
def node_one(state: State):
    print(f"입력한 배열의 길이: {len(state["words"])}")

def node_two(word: str):
    return {
        "output": [
            {
                "word": word,
                "letters": len(word)
            }
        ]
    }

def abc(state: State):
    words = state["words"]
    return [Send(node="node_two", arg=word) for word in words]

In [4]:
graph_builder = StateGraph(State)

graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)

graph_builder.add_edge(START, "node_one")
graph_builder.add_conditional_edges("node_one", abc, ["node_two"])
graph_builder.add_edge("node_two", END)

graph = graph_builder.compile()

In [5]:
graph.invoke(
    {
        "words": ["hello", "my_name", "is", "jaehyeon"],
    }
)

입력한 배열의 길이: 4


{'words': ['hello', 'my_name', 'is', 'jaehyeon'],
 'output': [{'word': 'hello', 'letters': 5},
  {'word': 'my_name', 'letters': 7},
  {'word': 'is', 'letters': 2},
  {'word': 'jaehyeon', 'letters': 8}]}